In [ ]:
include("../../src/Yggdrasil.jl")
using .Yggdrasil

# Example description

This notebook has examples on how to plot line data in a single figure (no subplots). 

First, the example shows how to create two lines. Then data is loaded from a snapshot. The data is then interpolated to the lines. 
Next, the lines are plottet individually.

Finally, the example shows how lines can be plotted together in a single figure. 

In [ ]:
midpoint = [16.27207,  0.0, 6.1138]
û     = [0.707, 0.0, -0.68]
û_perp = [û[1], û[2], -û[3]]
num_points = 50

n̂     = [0.0, 1.0, 0.0]
half_length = 0.3 # on either side of the line
half_width  = 0.2 # on either side of the line
n_perp = 50;

par_vals = range(-half_length, half_length, length=num_points)
line_par= get_line(midpoint,
                  û,
                  half_length,
                  num_points);


perp_vals = range(-half_length, half_length, length=num_points)
line_perp = get_line(midpoint,
                  û_perp,
                  half_length,
                  num_points);


In [ ]:
#---------------- define whhat data to load ----------------
data_folder = "../example_data/test_01/"    # experiment folder 
snap = 105                                  # snapshot number to load
ref_level = 3                               # load from the second level (refinement level 1)
load_pic = false;                           # if true only PIC patches are loaded, otherwise load MHD patches

#----------- load meta information ---------------------------
Snap_meta = read_snapshot(data_folder, snap)   # Load initial snapshot meta
IDX = Snap_meta.IDX
level_min = Snap_meta.LEVELMIN
level = level_min + ref_level     

#----------------- get area to load --------------------------
patches = 6
llc, urc, llc_pos, urc_pos  = get_area(Snap_meta, midpoint, patches, level);

#-------------- get coordiates ------------------------------
drop_dims = false
x,y,z,ds = get_xyz(Snap_meta, drop_dims, level; llc=llc, urc=urc); # for a subdomain defined by lower left corner (llc) and upper right corner (urc)
x_vals = x[:,1, 1]   # 1D arrays for interpolation
y_vals = y[1, :, 1]  # 1D arrays for interpolation
z_vals = z[1, 1, :]; # 1D arrays for interpolation


#----------------- load density for snapshot -------------------
data_rho = load_snapshot(Snap_meta, load_pic, llc, urc, "rho", use_level=level);

In [ ]:
#----------------- interpolate along parallel and perpendicular lines -------------------
rho_par =  interpolate_to_line(data_rho, x_vals, y_vals, z_vals, line_par);
rho_perp = interpolate_to_line(data_rho, x_vals, y_vals, z_vals, line_perp);

In [ ]:
#----------------- simplest way to plot -------------------
plot_line(rho_par)

In [ ]:
plot_line(rho_perp;
    x_vals = perp_vals,
    xlabel = "d_perp [ Mm ]",
    ylabel = "Density [ code units ]",
    title = "Density along perpendicular line",
    marker = "o",
    color = :blue,
    label = "density",
    legend = true
)

## Plotting multiple lines in single figure

In [ ]:
#---------------- combine lines to a single 2D matrix for plotting -------------------
comb = zeros(num_points, 2)
comb[:, 1] = rho_par
comb[:, 2] = rho_perp;

#----------------- plot both lines in a single figure -------------------
# here all lines are plotted. If only certain lines should be plotted
#                             use the `indices` keyword argument
plot_line(comb)

In [ ]:
#----------- althernative plotting with some extra options -----------
#- here the data is passed as a vector of vectors, where each vector corresponds to a line
plot_line([rho_par, rho_perp];
    x_all = [par_vals, perp_vals],
    label_all = ["rho_par", "rho_perp"],
    title = "Density along parallel and perpendicular lines",
    xlabel = "d [ Mm ]",
    ylabel = "Density [ code units ]"
)